In [ ]:
import json

with open('dataset.json', 'r') as dataset_file:
    dataset: list[dict[str]] = json.load(dataset_file)

# We are only interested in commits, not issues
dataset = [item for item in dataset if item['type'] == 'commit']

In [ ]:
from github import Github, Auth

with open('TOKEN.txt', 'r') as token_file:
    token = token_file.readline().strip()
    gh = Github(auth=Auth.Token(token))

print(f'Using GitHub as', gh.get_user().login)

In [ ]:
from github.Repository import Repository

from datetime import datetime, UTC
import os
import time

repos: dict[str, Repository] = {}

commit_diffs: list[dict[str]]
if os.path.exists('diffs.json'):
    with open('diffs.json', 'r') as diffs_file:
        commit_diffs = json.load(diffs_file)
else:
    commit_diffs = []

commit_errors: dict[str, str]
if os.path.exists('errors.json'):
    with open('errors.json', 'r') as errors_file:
        commit_errors = json.load(errors_file)
else:
    commit_errors = {}

diffs_file = open('diffs.json', 'w')
errors_file = open('errors.json', 'w')

for i, item in enumerate(dataset):
    _, _, _, owner, name, _, sha = item['url'].split('/')

    repo_id = f'{owner}/{name}'
    commit_url = item['url']

    try:
        # Skip already collected commits
        if not any(diff_item['url'] == commit_url for diff_item in commit_diffs):
            # Cache repo objects
            if repo_id not in repos:
                repos[repo_id] = gh.get_repo(repo_id)
            repo = repos[repo_id]

            commit = repo.get_commit(sha)

            if 'link' in commit.raw_headers:
                raise Exception('Commit is paginated')

            files = [{
                'filename': f.filename,
                'additions': f.additions,
                'deletions': f.deletions,
                'changes': f.changes,
                'status': f.status,
                'patch': f.patch,
            } for f in commit.files]

            commit_diffs.append({
                'url': item['url'],
                'files': files,
                'existing_codes': item['codes'],
                'codes': [],
                'notes': '',
            })
    except Exception as e:
        print(f'Error fetching commit {commit_url}:', e)
        commit_errors[commit_url] = str(e)

    with open('diffs.json', 'w') as diffs_file:
        json.dump(commit_diffs, diffs_file, indent=2)

    with open('errors.json', 'w') as errors_file:
        json.dump(commit_errors, errors_file, indent=2)

    if gh.rate_limiting[0] <= 1:
        now = datetime.now(tz=UTC)
        reset = datetime.fromtimestamp(gh.rate_limiting_resettime, tz=UTC)

        # If we hit the rate limit, sleep until the reset time (with a 60s buffer)
        time.sleep(float((reset - now).seconds + 60))

    if i % 10 == 0:
        print(f'{i}/{len(dataset)} commits processed, rate limit:', gh.rate_limiting, 'reset:', gh.rate_limiting_resettime)

print('Done')